In [ ]:
import numpy as np
from scipy.stats import zscore
import  pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
genome_names=['Africa','Ash1','Bengali' ,'Gujarat','Han1', 'hg38', 'KIn2_final'  ,'PJL_ITU', 'PJL_pat' ,'PR1', 'ITU']
chromosomes= [str(i) for i in range (1,23)] + ['X']
print (len(genome_names))

In [ ]:
for chr_name in chromosomes:
    for genome in genome_names:
        df_final = pd.DataFrame()
        #Blast output named as <genome>_chr<number>_t2t.out6
        file_path= f"<path_to_blast_input_dir>/{genome}/{genome}_chr{chr_name}_t2t.out6"
        
        cols = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen','qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
        #check if file exists
        if not os.path.exists(file_path):
            print (f"File not found: {file_path}. Skipping chr{chr_name} for {genome}")
            continue

        #read the blast file
        blast_df = pd.read_csv(file_path, sep="\t" , names=cols, header=None)
        
        #Filter the blast for percentage identity >= 98%
        blast_df_filt=blast_df[blast_df['pident'] >= 98]

        #pull out only the marker start and the subject start position
        #change str.extract(r'_(\d+)-')according to the query id first column in blast file
        df_final['Markers'] = blast_df_filt['qseqid'].str.extract(r'_(\d+)-')[0].astype(int)  
        print(df_final['Markers'])
        df_final[f'{genome}'] = blast_df_filt['sstart']
        df_final['send'] = blast_df_filt['send']

        reverse_count = (df_final[f'{genome}'] > df_final['send']).sum()
        total_count = len(df_final)

        if (reverse_count / total_count >= 0.49):
            df_final[f'{genome}']= sorted(df_final[f'{genome}'].values)


        #drop ['send']
        df_final=df_final.drop(columns=['send'])

        #write to file
        output_path=f"<path_to_output_dir>/{genome}/{genome}_chr{chr_name}_t2t_plot_new.txt"
        
        df_final.to_csv(output_path, sep='\t', index =False, header =None)
        print(f"Output for plotting chr{chr_name} of {genome} is saved at {output_path}.")

        #Empty the dataframe for next chromosome and genome
        df_final=pd.DataFrame()

In [ ]:
#Make a matrix chromosome wise by combining all genomes into one. 
for chr_name in chromosomes:
    
    combined_df = None
    
    for genome in genome_names:
        # Construct the file path
        file_path = f'<path_to_ouput_dir>/{genome}/{genome}_chr{chr_name}_t2t_plot_new.txt'
                
        # Check if the file exists before reading
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}. Skipping {genome} for chr{chr_name}")
            continue
        
        # Read the file for the current genome and chromosome
        chr_df = pd.read_csv(file_path, sep='\t', names=['Markers', genome])
        
        # Print first few rows of the file for debugging
        #print(f"File {file_path} loaded successfully. First rows of data for {genome}:\n", chr_df.head())

        # Merge the current genome data into the combined DataFrame
        if combined_df is None:
            combined_df = chr_df
        else:
            combined_df = pd.merge(combined_df, chr_df, on='Markers', how='outer')

    
    # Save the combined data for the current chromosome in a directory named Combined_plots
    combined_file_path = f'<path_to_ouput_dir>/Combined_plots/all_genome_chr{chr_name}.txt'
    
    

    combined_df.to_csv(combined_file_path, sep='\t', index=False)

    print(f"Combined file saved at: {combined_file_path}")

In [ ]:
for chr_name in chromosomes:
    # Read the combined file
    combined_file_path = f'<path_to_ouput_dir>/Combined_plots/all_genome_chr{chr_name}.txt'
    
    combined_df = pd.read_csv(combined_file_path, sep='\t')

    start_points={}

    for genome in genome_names:
        all_points=combined_df[genome].dropna()

        if all_points.empty:
            continue

        midpoint= len(all_points) // 2
        check_window=20
        start_slice= all_points.iloc[max(0, midpoint- check_window //2) : midpoint +check_window //2+1]

        if not start_slice.empty:
            start_points[genome]=start_slice.mean()

    median_start= np.median(list(start_points.values()))

    threshold=2_000_000

    for i,genome in enumerate (genome_names, start=1):
        if genome not in start_points:
            continue
        start_diff= start_points[genome]- median_start


        if start_diff > threshold:
            combined_df[genome]= combined_df[genome] - start_diff
            print(f"The genome {genome} was subtracted by {start_diff}")
   
    # Iteratively add 2M, 4M, 6M, etc. to columns from 2nd to 11th
    for i in range(1, len(genome_names)+1):  # Adjusting for zero-based index (1 means second column, 10 means 11th column)
        combined_df.iloc[:, i] += i * 5_550_000  # Add 2M, 4M, 6M, etc. Can be changed for each chromosome according to the distance between genomes seen in the plots 
       
    #Remove outlier points
    for genome in genome_names:
        # Calculate Z-scores for the genome column, treating blanks as missing
        zscores = zscore(combined_df[genome].astype(float), nan_policy='omit')
    
        # Replace outliers with an empty string
        combined_df.loc[abs(zscores) > 3, genome] = np.nan

        combined_df.loc[combined_df[genome] < 0, genome] = np.nan
    
    # Write the modified DataFrame back to the same file
    combined_df.to_csv(combined_file_path, sep='\t', index=False)

    print(f"Updated file saved at: {combined_file_path}")

    # Plot a dot plot for the chromosome
    plt.figure(figsize=(12, 7))  # Adjust figure size as needed
    
    
    
    for genome in genome_names:
            if genome == 'KIn2_final':
                # Plot 'KIn2_final' with larger size and diamond shape
                plt.scatter(combined_df['Markers'], combined_df[genome], label=genome, s=5, alpha=0.7, marker='D', color='deeppink')
            else:
                # Plot other genomes with smaller size and default circle shape
                plt.scatter(combined_df['Markers'], combined_df[genome], label=genome, s=2.5, alpha=0.7)


    # Labels and title
    plt.xlabel('Markers', fontsize='12')
    plt.ylabel('Genomes', fontsize='12')
    plt.title (f'Chromosome{chr_name}', fontsize='12')

    # Add grid lines for better visibility
    plt.grid(True)

    # Set x and y axis limits (if needed)
    plt.xlim([combined_df['Markers'].min(), combined_df['Markers'].max()])
    plt.ylim([combined_df[genome_names].min().min(), combined_df[genome_names].max().max()])

    # Optional: Add legend
    # Get legend handles and labels
    handles, labels = plt.gca().get_legend_handles_labels()

    # Reverse the handles and labels if needed
    handles, labels = reversed(handles), reversed(labels)

    # Rename the labels as desired
    custom_labels = ['ITU1', 'PR1', 'PJL1.v1', 'PJL1.v2', 'KIn1.v2', 'Hg38' ,'Han1', 'GIH1', 'BIB1', 'Ash1' ,'GWD1' ]
    plt.legend(handles,custom_labels, loc='lower right',markerscale=1.5,fontsize='8')
    
    # Save as PNG in a dir named Plot_images
    plt.savefig(f"<path_to_ouput_dir>/Plot_images/{chr_name}.png", dpi=300, bbox_inches='tight')
    
    # Show plot
    plt.show()
